# 正則化手法の調査

## 手法の概要

scikit-learn 公式ドキュメントに、解説があります。

L1正則化
http://scikit-learn.org/stable/modules/linear_model.html#lasso

L2正則化
http://scikit-learn.org/stable/modules/linear_model.html#ridge-regression

一般的な認識としましては、

L1正則化・・・次元削除に使用される（＝特徴の数を削減する）

L2正則化・・・モデルの過学習を抑えて汎化させるために使用される（＝予測に大きな影響を与えているパラメータにペナルティを課すことで過学習を抑える）

といったところかと存じます。

今回はL2正則化にスポットを当てて調査するものといたします。

## 調査の方向性

今回調査では、モデルを簡単にするため、前回調査で使用した TF ベクターを training_set として使用するようにします。

TFベクターとは・・・scikit-learn の入力となる、下記のような配列です（<a href="../LDA_in_scikit_learn_05.ipynb">前回調査</a>で使用した tf_vector_array と同じ）

In [1]:
X = [
    [1, 0, 1, 0, 1], # ここに単語出現数が入る
    [2, 1, 1, 0, 1],
    [0, 1, 2, 1, 0],
    [2, 0, 1, 0, 1],
    ...
    ]

y = [1, 2, 3, 2, ...] # ここにクラスIDが入る

すなわち x の各要素（＝出現単語じたい）を、回帰予測モデルで言うところの「パラメータ」に置き換えれば、正則化手法が使えるのでは？　という仮説のもとでの試みになります。

正則化を実行する際、下記のような class_weight という配列を用意し（←これをどうやって、負例データから作成するかは別途要・調査ですが・・・）、例えばロジスティック回帰のインプットとして指定した上で予測する・・・という仕組みになるかと考えます。

In [2]:
class_weight = [1, 0.2, 0.8, 1, ...] # ここにクラスごとの重み付けが入る

## scikit-learn のロジスティック回帰における正則化

まずは、scikit-learn のロジスティック回帰モジュールを使用する前提で調査してみます。

scikit-learn の sklearn.linear_model.LogisticRegression に関する記述：

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html


パラメータごとにペナルティを課すことが可能か否か（可能な場合はそのための方法）が調査観点になります。

### (1) コンストラクターのパラメータ

<b>__init__</b> 関数の関連するパラメータは下記の通りと考えられます。

- penalty : str, ‘l1’ or ‘l2’, default: ‘l2’

  デフォルト＝L2正則化とのこと。
  
  
- C : float, default: 1.0　--->正則化の効きを表す数値

  値が小さい方が正則化の効きが強いとのこと。
  
  たとえば C=0.1 は C=1 より効きが強い＝ペナルティを課しやすい、ということになるかと存じます。


- class_weight : dict or ‘balanced’, default: None　---> ここにクラス（＝マイオペにおいては質問に対する回答）ごとの重み付けを与える。

  ここで指定した class_weight が、ペナルティとして効いてくるのかどうかの説明はありませんが・・・おそらくそうなのだと考えております。
  
  為念で、実験の上、確認するしかなさそうです。


- solver : {‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’}, default: ‘liblinear’　--->採用するアルゴリズムを選択。

  L2正則化を扱うのは newton-cg、lbfgs、sag のいずれかとのこと。



ちなみに・・・<b>C</b> や <b>solver</b> につきましては、どれが適しているのかは不明なので、グリッドサーチ実行により、最適なものを選択することになるかもしれません。

### (2) 学習時のパラメータ

<b>fit</b> 関数の関連するパラメータは下記の通りと考えられます。

- sample_weight : array-like, shape (n_samples,) optional　--->fit 関数の第３引数。

  ここで指定した sample_weight は、サンプル（＝マイオペにおいては質問文）ごとの重みということになります。

### (3) 正則化パラメータをどうやって決めるのか？

収集された負例データから、class_weight や sample_weight といった値をどうやって導出するのか、というロジックが現時点では？？？です。

別途、検討しなければなりません。

ひとまず、ロジスティック回帰におけるL2正則化の動きを確認する為、ここではあらかじめ決めておいた固定値を使用するものとします。

### (4) パラメータと効果についての確認

実際に、scikit-learn の LogisticRegression に対し、class_weight、sample_weight をパラメータ指定して動作確認いたしました。

結果は<b><a href="04.ipynb">こちらのページ</a></b>に掲載しております（2017/02/24改訂）。

- class_weight（コンストラクターのパラメータ）

  クラスごとに重み付けを与えることができる （クラス＝質問文から選択された回答文と等価）


- sample_weight（学習時のパラメータ）

  訓練データごとに重み付けを与えることができる （訓練データ＝質問文１件と等価）
  
  
（注）改訂前の結果は<b><a href="02.ipynb">こちらのページ</a></b>に掲載しております。